In [2]:
# !pip3 install -U polygon-api-client
# !pip3 install setuptools
# !pip3 install pandas_ta
from polygon import RESTClient
import numpy as np
import pandas as pd
import pandas_ta as ta

In [3]:
polygon_api_key   = 'ZV6Xy3SWzsJ2xawzRp0JXlmUbbIHePWF'#'_jyyfwbAshFTAtdM3jaZIu9JnKLv7npG'
client            = RESTClient(api_key=polygon_api_key)
ticker            = "AAPL"
from_date         = "2019-01-01"
to_date           = "2024-01-13"
max_date          = pd.to_datetime(to_date) - pd.Timedelta('1 day')

def get_df_price(client, ticker, from_date, to_date, timespan, multiplier, max_date):
    aggs = []
    for a in client.list_aggs(ticker=ticker, multiplier=multiplier, timespan=timespan, from_=from_date, to=to_date, limit=50000):
        aggs.append(a)
    df = pd.DataFrame(aggs)
    df.timestamp = pd.to_datetime(df.timestamp, unit='ms')
    df = df[df.timestamp <= max_date]
    df.set_index('timestamp', inplace=True)
    return df


df_hour  = get_df_price(client, ticker, from_date, to_date, "hour", 1, max_date)
df_day   = get_df_price(client, ticker, from_date, to_date, "day", 1, max_date)
df_4hour = get_df_price(client, ticker, from_date, to_date, "hour", 4, max_date)
df_15min = get_df_price(client, ticker, from_date, to_date, "minute", 15, max_date)



In [4]:
df_hour.ta.macd(fast=12, slow=26, signal=9)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
timestamp,,,
2019-03-05 09:00:00,NaN,NaN,NaN
2019-03-05 10:00:00,NaN,NaN,NaN
2019-03-05 11:00:00,NaN,NaN,NaN
2019-03-05 12:00:00,NaN,NaN,NaN
2019-03-05 13:00:00,NaN,NaN,NaN
...,...,...,...
2024-01-11 20:00:00,0.010705,-0.152176,0.162881
2024-01-11 21:00:00,0.030936,-0.105556,0.136492
2024-01-11 22:00:00,0.051221,-0.068217,0.119438


In [5]:




class HelperFunctions:

    @staticmethod
    def _determine_multiplier(num):
        decimal_places = str(num).find('.')
        multipliers = { 2: 1000, 3: 100, 4: 10, 5: 1, 
                       6: 0.1, 7: 0.01, 8: 0.001, 9: 0.0001, 10: 0.00001
        }
        return multipliers.get(decimal_places, 10000)

    @staticmethod
    def pip(num1, num2):
        multiplier = HelperFunctions._determine_multiplier(num1)
        return [num1 * multiplier, num2 * multiplier]

    @staticmethod
    def pipreverse(num, num2):
        multiplier = 1 / HelperFunctions._determine_multiplier(num)
        return num2 * multiplier

    @staticmethod
    def pipdiffy(price, num1):
        multiplier = HelperFunctions._determine_multiplier(price)
        return num1 * multiplier

    @staticmethod
    def valdiff(history, num1, num2):
        # Calculate the value difference as a decimal percentage of the floor to ceiling.
        history_close = history['close'].values
        floor = min(history_close)
        ceil = max(history_close)
        valdiffer = ceil - floor
        diff = abs(num1 - num2)
        valuediff = diff / valdiffer
        return valuediff
    
    @staticmethod
    def volatility(history_df):
        r = history_df.ta.rsi(length=14).values

        # Get the last value of the RSI
        q = r[-1]

        # Calculate the difference based on the RSI value
        diff = q - 0 if q > 50 else 100 - q

        # Calculate the volatility equation
        difference = diff / 100
        equation = abs(100 * np.sin(difference)) / abs(100 * np.sin(1.05))
        return equation
    
    @staticmethod
    def trend(history_df):
        history = history_df['close'].values
        if history[-1] > history[-2] and history[-2] > history[-3]:
            return True
        if history[-1] < history[-2] and history[-2] < history[-3]:
            return False
        # TODO: What if the price is the same as the previous price or the conditions fail? 

    @staticmethod
    def rsi(history_df):
        history = history_df['close'].values
        q = history_df.ta.rsi(length=14)
        qlast = q.iloc[-1]

        # Create a copy of the history without the last value
        history2 = history[:-1]
        history2_df = pd.DataFrame(history2, columns=['close'])
        r = history2_df.ta.rsi(length=14)
        rlast = r.iloc[-1]

        if qlast > rlast:
            return True
        if rlast > qlast:
            return False
        
    @staticmethod
    def macd(history):
        # Calculate the MACD line, signal line, and histogram
        macd = history.ta.macd(fast=12, slow=26, signal=9)
        
        # Get the last two values of the histogram
        qlast = macd.iloc[-1]['MACDh_12_26_9']
        rlast = macd.iloc[-2]['MACDh_12_26_9']
        
        # Determine the trend based on the histogram
        if qlast > rlast:
            return True
        elif rlast > qlast:
            return False
        else:
            return None  # No clear trend
        
    @staticmethod
    def obv(history):
        # Calculate the RSI based on the closing prices
        rsi = history.ta.rsi(length=14)
        rsi = rsi.to_frame().rename(columns={'RSI_14': 'close'})
        
        # Calculate the 8-period EMA of the RSI
        ema_fast = rsi.ta.ema(length=8)
        
        # Calculate the 14-period EMA of the RSI
        ema_slow = rsi.ta.ema(length=14)
        
        # Get the last values of the fast and slow EMAs
        ema_fast_last = ema_fast.iloc[-1]
        ema_slow_last = ema_slow.iloc[-1]
        
        # Determine the trend based on the EMAs
        if ema_fast_last > ema_slow_last:
            return True
        elif ema_slow_last >= ema_fast_last:
            return False

    @staticmethod
    def roc(history_df):
        # Calculate the Rate of Change (ROC) for the given history
        roc_values = history_df.ta.roc(length=10)
        
        # Get the last two ROC values
        roc_last = roc_values.iloc[-1]
        roc_second_last = roc_values.iloc[-2]
        
        # Determine the trend based on the ROC values
        if roc_last > roc_second_last:
            return True
        elif roc_second_last >= roc_last:
            return False
        

    @staticmethod
    def consolidation(history_df):
        # Determine consolidation based on volatility
        return HelperFunctions.volatility(history_df) <= 0.618
    

    @staticmethod
    def closest(num1, finlevs, biggersupres, price):
        # Find the closest support and resistance level to the given price
        values = finlevs + biggersupres
        valdiffgreater = []
        valdiffless = []

        for value in values:
            if num1 < value:
                valdiffgreater.append(abs(num1 - value))
            if num1 > value:
                valdiffless.append(abs(num1 - value))

        if valdiffless:
            closestbelow = price - min(valdiffless)
        else:
            closestbelow = float('inf')

        if valdiffgreater:
            closestabove = price + min(valdiffgreater)
        else:
            closestabove = float('inf')
        
        if closestabove == float('inf') and closestbelow == float('inf'):
            raise ValueError('No support or resistance levels found')

        return min(closestbelow, closestabove)


    @staticmethod
    def keylev(history, price, finlevs, biggersupres,):
        # Check if the current price is close to a key level
        # TODO: Checkc why price and num1 are the same for js implementation???
        return HelperFunctions.valdiff(history, price, HelperFunctions.closest(price, finlevs, biggersupres, price)) < 0.1

    @staticmethod
    def ema(history):
        q = history.ta.ema(8)
        r = history.ta.ema(14)
        return q.values[-1] > r.values[-1]

    @staticmethod
    def supreslevs(history, price_diff_pct=0.05):
        history_close = history['close'].values
        ceiling = max(history_close)
        floor = min(history_close)
        difference = ceiling - floor

        # Identify levels based on the relative position in the price range.
        levels = [(price - floor) / difference for price in history_close]

        # Round levels to three decimal places.
        levels = [round(level, 3) for level in levels]

        # Count occurrences of each level.
        level_counts = {level: levels.count(level) for level in set(levels)}

        # Filter levels that occur more than 3 times.
        significant_levels = [level for level, count in level_counts.items() if count > 3]

        # Calculate price levels from significant levels.
        final_levs = [(level * difference) + floor for level in significant_levels]

        # Determine support and resistance levels based on the current price.
        price = history_close[-1]  # Current price (last price in history)
        smaller = [lev for lev in final_levs if lev < price]
        larger = [lev for lev in final_levs if lev > price]

        # Filter levels based on their percentage distance from the current price.
        smaller_two = [lev for lev in smaller if HelperFunctions.valdiff(history, price, lev) > price_diff_pct]
        larger_two  = [lev for lev in larger if HelperFunctions.valdiff(history, price, lev) > price_diff_pct]
        # Ensure there is at least one level below and above the current price.
        # TODO: Implement stoploss 
        # if not smaller_two:
        #     smaller_two.append(price - HelperFunctions.pipreverse(price, HelperFunctions.pipdiffy(price, HelperFunctions.stoploss())))
        # if not larger_two:
        #     larger_two.append(price + HelperFunctions.pipreverse(price, HelperFunctions.pipdiffy(price, HelperFunctions.stoploss())))

        # Find the closest levels in the filtered lists.
        support = price - min([abs(lev - price) for lev in smaller_two])
        resistance = price + min([abs(lev - price) for lev in larger_two])

        # Return support and resistance levels.
        return support, resistance, final_levs
    
    @staticmethod
    def consolidationtwo(history, bbands_length=10, bbands_std=1, atr_length=8, sma_length=14):
        history_close = history['close'].values
        # Calculate historical maximum and minimum prices and their difference
        hist_max = max(history_close)
        hist_min = min(history_close)
        hist_diff = hist_max - hist_min

        # Calculate Bollinger Bands with a period of 10 and standard deviation of 1
        q = history[['close']].ta.bbands(length=bbands_length, std=bbands_std)

        # Calculate True Range with a period of 8
        n = history.ta.true_range(atr_length).values

        # Calculate Simple Moving Averages (SMAs) for lower and upper Bollinger Bands
        smmas =  q[[f'BBL_{bbands_length}_1.0']].rename(columns={f'BBL_{bbands_length}_1.0': 'close'} ).ta.sma(sma_length).values
        smmass = q[[f'BBU_{bbands_length}_1.0']].rename(columns={f'BBU_{bbands_length}_1.0': 'close'} ).ta.sma(sma_length).values

        # Keep the midpoint just in case (not used in this implementation)
        smmasss = q[[f'BBM_{bbands_length}_1.0']].rename(columns={f'BBM_{bbands_length}_1.0': 'close'} ).ta.sma(sma_length).values

        # Get the last values of the SMAs for lower and upper Bollinger Bands
        smmas_last  = smmas[-1]
        smmass_last = smmass[-1]

        # Calculate the difference between the last values of the upper and lower SMAs
        sma_diff    = smmass_last - smmas_last

        # Calculate the differences between consecutive True Range values
        ndiff_one = n[-1] - n[-2]
        ndiff_two = n[-2] - n[-3]

        # Define a benchmark value based on historical price difference
        benchmark = 0.025 * hist_diff

        # Check if the difference between the upper and lower SMAs is greater than the benchmark
        # and if the True Range is increasing
        if sma_diff > benchmark and (n[-1] > n[-2] and ndiff_one > ndiff_two):
            return False
        else:
            return True


    @staticmethod
    def overall(history, extended_history_df):
        price            = history['close'].values[-1]
        extended_history = extended_history_df['close'].values
        # Calculate maximum and minimum prices in the extended history
        max_price = max(extended_history)
        min_price = min(extended_history)

        # Define a buffer based on the price range
        buffer = (max_price - min_price) * 0.05

        # Define upper and lower price limits for the study range
        lower = price - buffer
        upper = price + buffer
        price_range = [lower, upper]

        # Create a list of prices within the study range
        study_list = [[val, extended_history[val]] for val in range(len(extended_history)) if lower <= extended_history[val] <= upper]

        # Perform analysis on the study list and return the result
        result = HelperFunctions.analysis(history, study_list, extended_history, price_range)
        overall, rejection_zones = result
        return overall, rejection_zones 

    @staticmethod
    def analysis(history, cases, extended_history, price_range):
        price = history['close'].values[-1]
        # Normalize the historical price data
        hist_norm = history['close'].values
        norm_diff = (max(hist_norm) - min(hist_norm)) * 0.025

        # Convert extended_history to a DataFrame and calculate Bollinger Bands and SMAs
        q = history[['close']].ta.bbands(length=10, std=1)
        n = history.ta.true_range(length=8).values
        smmas = q[['BBL_10_1.0']].rename(columns={'BBL_10_1.0': 'close'}).ta.sma(14).values
        smmass = q[['BBU_10_1.0']].rename(columns={'BBU_10_1.0': 'close'}).ta.sma(14).values
        smmasss = q[['BBM_10_1.0']].rename(columns={'BBM_10_1.0': 'close'}).ta.sma(14).values

        # Calculate the midpoint of the price range
        hist_diff = (price_range[1] - price_range[0]) / 2

        # Define a benchmark value based on the midpoint of the price range
        benchmark = 0.025 * hist_diff

        # Initialize a list for fractals and a counter for rejection zones
        fractals = []
        rejection = 0
        rejection_zones = []

        # Identify fractals within the cases
        for val in cases:
            fractals.append(val[0])

        # Analyze fractals for rejection zones
        for val in fractals:
            min_count = 0
            max_count = 0
            for value in range(3):
                if 1 < val < (len(extended_history) - 2):
                    if extended_history[val] > extended_history[val - value]:
                        max_count += 1
                    if extended_history[val] > extended_history[val + value]:
                        max_count += 1
                    if extended_history[val] < extended_history[val - value]:
                        min_count += 1
                    if extended_history[val] < extended_history[val + value]:
                        min_count += 1
            if min_count or max_count > 4:
                rejection += 1
                if not fractals:
                    fractals.append(0)
                    rejection_zones.append(fractals[0])
                else:
                    rejection_zones.append(extended_history[val])

        # Handle cases with no rejection zones
        if not rejection_zones:
            rejection_zones.append(price)

        # Return True if there are less than or equal to 2 rejection zones, otherwise return False
        return rejection <= 2, rejection_zones

    @staticmethod
    def closesttwo(num1, price, finlevs, biggersupres):
        values = finlevs + biggersupres
        valdiffgreater = [abs(num1 - value) for value in values if num1 < value]
        valdiffless = [abs(num1 - value) for value in values if num1 > value]
        closestbelow = price - min(valdiffless) if valdiffless else None
        closestabove = price + min(valdiffgreater) if valdiffgreater else None
        return closestbelow, closestabove
    
    @staticmethod
    def stoploss(history_df, finlevs, biggersupres, pot_buy, pot_sell):
        highs = history_df['high'].values[-31:]
        lows = history_df['low'].values[-31:]
        diffs = [abs(highs[i] - lows[i]) for i in range(len(highs))]
        total_diff = sum(diffs)
        final_diff = (total_diff / 30) * 1.382 if HelperFunctions.volatility(history_df) > 0.618 else total_diff / 30

        price = history_df['close'].values[-1]
    
        new_sl = 0
        if pot_buy:
            diff_price = price - final_diff
            closest_below, _ = HelperFunctions.closesttwo(diff_price, price, finlevs,  biggersupres)
            if closest_below is None:
                new_sl = price - (final_diff * 3.618)
            else:
                new_sl = diff_price - (0.786 * (diff_price - closest_below))
        elif pot_sell:
            diff_price = final_diff + price
            _, closest_above = HelperFunctions.closesttwo(diff_price, price, finlevs, biggersupres)
            if closest_above is None:
                new_sl = price + (final_diff * 3.618)
            else:
                new_sl = diff_price + (0.786 * (closest_above - diff_price))
                
        # TODO: Can both pot_buy and not pot_buy be true at the same time?
        # TODO: Add case for when pot_buy and pot_sell are both false
        # TODO: Recheck logic for setting new_sl


        return final_diff, new_sl
    
    @staticmethod
    def tpvariation(history_df, tp, finlevs, biggersupres, price, buy_pos, sell_pos):
        values = finlevs + biggersupres
        valdiffgreater = []
        valdiffless = []
        closesttp = tp
        filteredvaldiff = []
        nexttp = 0
        referenceval = 0
        volval = HelperFunctions.volatility(history_df)

        if buy_pos:
            valdiffgreater = [abs(price - value) for value in values if price < value]
            filteredvaldiff = list(set(valdiffgreater))
            filteredvaldiff = [val for val in filteredvaldiff if val >= closesttp - price]

            if volval > 0.618:
                tp = price + (abs(price - tp) * 1.382)

                if filteredvaldiff and min(filteredvaldiff) < float('inf') and min(filteredvaldiff) > 0 and min(filteredvaldiff) != price:
                    nexttp = price + (abs(price - min(filteredvaldiff)) * 1.382)
                else:
                    nexttp = price + ((tp - price) * 1.618)
            else:
                if filteredvaldiff and min(filteredvaldiff) < float('inf') and min(filteredvaldiff) > 0 and min(filteredvaldiff) != price:
                    nexttp = price + min(filteredvaldiff)
                else:
                    nexttp = price + ((tp - price) * 1.382)

        if sell_pos:
            valdiffless = [abs(price - value) for value in values if price > value]
            filteredvaldiff = list(set(valdiffless))
            filteredvaldiff = [val for val in filteredvaldiff if val >= price - closesttp]

            if volval > 0.618:
                tp = price - (abs(price - tp) * 1.382)
                if filteredvaldiff and min(filteredvaldiff) < float('inf') and min(filteredvaldiff) > 0 and min(filteredvaldiff) != price:
                    nexttp = price - (abs(price - min(filteredvaldiff)) * 1.382)
                else:
                    nexttp = price - ((price - tp) * 1.618)
            else:
                if filteredvaldiff and min(filteredvaldiff) < float('inf') and min(filteredvaldiff) > 0 and min(filteredvaldiff) != price:
                    nexttp = price - min(filteredvaldiff)
                else:
                    nexttp = price - ((price - tp) * 1.382)

        return tp, nexttp #nexttp is tptwo in js implementation
        # TODO: When is tp set? It is used in tpvariation but not set anywhere before in the code?

    @staticmethod
    def rejectionzoning(history_df, extended_history_df, pot_buy, pot_sell, tp):
        # Get the overall analysis and rejection zones
        _, rejection_zones = HelperFunctions.overall(history_df, extended_history_df)
        price = history_df['close'].values[-1]
        diffs = []

        # Calculate the differences between the price and rejection zones
        for val in rejection_zones:
            if pot_buy and price < val:
                diffs.append(val - price)
            if pot_sell and price > val:
                diffs.append(price - val)

        # Check if the minimum difference is less than the distance to the take profit
        if diffs and abs(min(diffs)) < abs(price - tp):
            pot_buy = False
            pot_sell = False
            return True
        else:
            return False

In [6]:
n_vals         = 1000 # TODO: How much historical data is required for every run?
price_diff_pct = 0.02 # TODO: How much percentage difference is required for a level to be considered significant? Why is it set to 0.05 initially?


support_hour, resistance_hour, final_levs_hour    = HelperFunctions.supreslevs(df_hour[-n_vals:], price_diff_pct)
support_day, resistance_day, final_levs_day       = HelperFunctions.supreslevs(df_day[-n_vals:], price_diff_pct)
support_4hour, resistance_4hour, final_levs_4hour = HelperFunctions.supreslevs(df_4hour[-n_vals:], price_diff_pct)

In [7]:
# Control Bigger Periods
totallevs    = final_levs_4hour + final_levs_day
biggersupres = totallevs

# TODO: In Js implementation, totallevs is an expanding list of final_levs_4hour and final_levs_day. Why do we need to keep track of all the levels?


In [8]:
consolidationtwo = HelperFunctions.consolidationtwo(df_hour[-n_vals:])
overall, rejection_zones = HelperFunctions.overall(df_hour[-n_vals:], df_hour[-n_vals*5:])
consolidation = HelperFunctions.consolidation(df_hour[-n_vals:])
keyLev = HelperFunctions.keylev(df_hour[-n_vals:], df_hour['close'].values[-1], final_levs_hour, biggersupres)

In [9]:
ema = HelperFunctions.ema(df_15min[-n_vals:])
obv = HelperFunctions.obv(df_15min[-n_vals:])
rsi = HelperFunctions.rsi(df_15min[-n_vals:])
trend = HelperFunctions.trend(df_15min[-n_vals:]) # TODO: Check case where trend is None
macd = HelperFunctions.macd(df_15min[-n_vals:])
roc = HelperFunctions.roc(df_15min[-n_vals:])
roc

False

In [11]:
final_diff, new_sl = HelperFunctions.stoploss(df_15min[-n_vals:], final_levs_hour, biggersupres, True, False)

In [10]:
# One_Hour_Nexus.controlSmallerPeriod()
    # stoploss()
    # tpvariation()
    # rejectionzoning()
# One_Hour_Functions.buy()

           
